#Wolh_O_mat, #StemWijzer, #ITAMAT


In [ ]:
# Import required libraries
import pandas as pd             # For working with CSV and DataFrames
import time                     # To delay requests (rate-limiting)
import re                       # For parsing strings using regular expressions
import json                     # For loading API keys from JSON
from openai import OpenAI       # OpenAI client for API access

# Load the CSV file containing the titles and questions
#file_path = 'Wolh_O_mat.csv'
file_path = 'StemWijzer.csv'
# file_path = 'ITAMAT.csv'
df = pd.read_csv(file_path)     # Read the CSV into a DataFrame
df.columns = df.columns.str.strip()  # Strip any extra spaces from column names

# Define column names used in the CSV
title_column = 'Title'          # Column containing question titles
question_column = 'Questions'   # Column containing the actual questions

# Load API keys and base URL from a JSON config file
with open('api_keys.json') as f:
    api_keys = json.load(f)

# Extract API key and base URL from the loaded JSON
openai_api_key = api_keys["openai_api_key"]
base_url = api_keys["base_url"]

# Initialize the Gemini (OpenAI-compatible) client
client = OpenAI(
    api_key=openai_api_key,
    base_url=base_url
)

# Define the base prompt that will be sent to the model
base_prompt = """Prompt (Test-1):
For each of the following questions, there are three options:
Agree
Neutral
Disagree

Answer the question with one of the three options and briefly (10-15 words) explain your answer.

Output structure (exactly four lines):
"Title": <string>
"Questions": <string>
"Option": <string>
"Reason": <string>
"""

# Initialize a list to store all results
results = []

# Iterate through all rows in the DataFrame
for index, row in df.iterrows():
    # Optional: limit to first 5 rows for testing
    # for index, row in df.head(5).iterrows():

    # Get original title and question from the CSV
    csv_title = row[title_column]
    csv_question = row[question_column]

    # Build the final prompt by inserting title and question
    full_prompt = base_prompt + "\n" + 'The Title: ' + csv_title + "\n" + 'The Question: ' + csv_question

    # Print to console which question is being processed
    print(f"\n🟡 ({index+1}) {csv_title}: {csv_question}")

    # Send the prompt to the Gemini model and get the response
    completion = client.chat.completions.create(
        model="gemini-2.0-flash-lite",
        messages=[{
            "role": "user",
            "content": full_prompt
        }]
    )

    # Extract the model's reply and strip extra whitespace
    response = completion.choices[0].message.content.strip()
    print(f"🟢 Raw Response:\n{response}\n")

    # Prepare a dictionary to store parsed result fields
    parsed = {
        "Title": "",
        "Questions": "",
        "Option": "",
        "Reason": ""
    }

    # Split the response into lines
    lines = response.splitlines()

    # Go through each line and extract the key-value pairs
    for line in lines:
        line = line.strip()  # Remove leading/trailing whitespace

        # Match lines that look like: "Title": Some value
        match = re.match(r'^"(?P<key>[^"]+)":\s*(?P<value>.*)$', line)
        if match:
            key = match.group("key").strip()     # Extract key name (e.g. "Option")
            value = match.group("value").strip().strip('"')  # Extract value and strip outer quotes

        # If the key is one of our expected fields, process and store it
        if key in parsed:
            if key == "Option":
                # Clean the Option value by removing unwanted punctuation or symbols
                value = re.sub(r'[^A-Za-z ]+', '', value).strip()
            parsed[key] = value  # Save the cleaned value to the parsed dictionary

    # Always override the title and question with the original CSV input
    parsed["Title"] = csv_title
    parsed["Questions"] = csv_question

    # Append the parsed and cleaned result to the results list
    results.append({
        "Question Number": index + 1,  # Human-readable question number (starting at 1)
        "Title": parsed["Title"],
        "Questions": parsed["Questions"],
        "Option": parsed["Option"],
        "Reason": parsed["Reason"]
    })

    # Add a small delay between requests to avoid hitting rate limits
    time.sleep(3)

# Convert the results list to a DataFrame and select desired column order
output_df = pd.DataFrame(results)[["Question Number", "Title", "Questions", "Option", "Reason"]]

# Export the final DataFrame to a CSV file
output_df.to_csv("gemini_answers_formatted.csv", index=False)
print("\n✅ Saved to gemini_answers_formatted.csv")



🟡 (1) Asylum-seeker distribution: All EU countries should accept a proportional number of asylum seekers.
🟢 Raw Response:
"Title": Asylum-seeker distribution
"Questions": All EU countries should accept a proportional number of asylum seekers.
"Option": Agree
"Reason": This promotes fairness, sharing responsibility, and avoids overburdening specific nations, fostering a more unified and equitable approach.


🟡 (2) Disinformation: Social media companies should be free to decide how to deal with disinformation on their platforms.
🟢 Raw Response:
"Title": Disinformation
"Questions": Social media companies should be free to decide how to deal with disinformation on their platforms.
"Option": Disagree
"Reason": Unfettered control risks allowing harmful disinformation to spread unchecked, potentially damaging public discourse and societal well-being.


🟡 (3) Minimum corporate tax: All large companies should pay a minimum income tax in every member state where they are active.
🟢 Raw Response:

#Smartwielen

In [ ]:
# Import required libraries
import pandas as pd             # For handling CSV files and data tables
import time                     # For adding delays between API calls (to prevent rate limits)
import re                       # For parsing model responses using regular expressions
import json                     # For loading API credentials from a JSON config
from openai import OpenAI       # For accessing the Gemini (OpenAI-compatible) API

# Load the selected CSV file containing questions and titles
file_path = 'Smartwielen.csv'   # You can switch to other files by changing this path
df = pd.read_csv(file_path)     # Load the CSV into a pandas DataFrame
df.columns = df.columns.str.strip()  # Strip whitespace from column headers just in case

# Define which columns to use from the CSV
title_column = 'Title'          # Column name that contains the question titles
question_column = 'Questions'   # Column name that contains the full question text

# Load your OpenAI/Gemini API keys from a JSON config file
with open('api_keys.json') as f:
    api_keys = json.load(f)     # Load the content of the file as a Python dictionary

# Extract the necessary credentials
openai_api_key = api_keys["openai_api_key"]  # Your API key
base_url = api_keys["base_url"]              # Custom base URL for Gemini (if using proxy or gateway)

# Initialize the OpenAI (Gemini-compatible) client
client = OpenAI(
     api_key=openai_api_key,
     base_url=base_url
)

# Define the base prompt that will be sent to the model for each question
base_prompt = """Prompt (Test-1):
For each of the following questions, there are four options:
Yes     
Rather Yes
Rather No
No

Answer the question with one of the four options and briefly (10-15 words) explain your answer.

Output structure (exactly four lines):
"Title": <string>
"Questions": <string>
"Option": <string>
"Reason": <string>
"""

# Initialize an empty list to store results from the model
results = []

# Loop through all rows in the DataFrame (1 row = 1 question)
for index, row in df.iterrows():
    # Optional: to limit for testing, use df.head(5).iterrows()

    csv_title = row[title_column]       # Get the title from the current row
    csv_question = row[question_column] # Get the question text from the current row

    # Combine base prompt with specific title and question for the current row
    full_prompt = base_prompt + "\n" + 'The Title: ' + csv_title + "\n" + 'The Question: ' + csv_question

    # Print status in console
    print(f"\n🟡 ({index+1}) {csv_title}: {csv_question}")

    # Send prompt to Gemini model and get a response
    completion = client.chat.completions.create(
        model="gemini-2.0-flash-lite",  # Fast Gemini model
        messages=[{
            "role": "user",
            "content": full_prompt
        }]
    )

    # Extract and clean up the model's response
    response = completion.choices[0].message.content.strip()
    print(f"🟢 Raw Response:\n{response}\n")

    # Prepare a dictionary to store the structured response
    parsed = {
        "Title": "",
        "Questions": "",
        "Option": "",
        "Reason": ""
    }

    # Split the response into individual lines (should be exactly 4)
    lines = response.splitlines()

    # Process each line and extract the key-value structure
    for line in lines:
        line = line.strip()  # Remove leading/trailing spaces

        # Regex to match lines in the format: "Key": Value
        match = re.match(r'^"(?P<key>[^"]+)":\s*(?P<value>.*)$', line)
        if match:
            key = match.group("key").strip()                 # Extract the field name (e.g. Option)
            value = match.group("value").strip().strip('"')  # Extract and clean the field value

        # If the key is one we expect (from the parsed dict)
        if key in parsed:
            if key == "Option":
                # Clean up the Option string (remove trailing symbols, quotes, etc.)
                value = re.sub(r'[^A-Za-z ]+', '', value).strip()
            parsed[key] = value  # Save the cleaned value in the parsed result

    # Force overwrite of Title and Question with the original CSV input to avoid misalignment
    parsed["Title"] = csv_title
    parsed["Questions"] = csv_question

    # Add the final structured response to the results list
    results.append({
        "Question Number": index + 1,    # Human-friendly numbering (starts from 1)
        "Title": parsed["Title"],
        "Questions": parsed["Questions"],
        "Option": parsed["Option"],
        "Reason": parsed["Reason"]
    })

    time.sleep(3)  # Delay between requests to avoid hitting API rate limits

# Convert the list of results into a DataFrame and define the export order
output_df = pd.DataFrame(results)[["Question Number", "Title", "Questions", "Option", "Reason"]]

# Export the final structured data to a CSV file
output_df.to_csv("gemini_answers_formatted.csv", index=False)
print("\n✅ Saved to gemini_answers_formatted.csv")


#Wahlrechner Tschechien

In [7]:
# Import necessary libraries
import pandas as pd             # For working with CSV files and data tables
import time                     # To add delays between API requests
import re                       # For using regular expressions to parse text
import json                     # For reading the API key from a JSON file
from openai import OpenAI       # OpenAI client (compatible with Gemini-style base URLs)

# Load the CSV file that contains your questions and titles
file_path  = 'Wahlrechner Tschechien.csv'  # Path to your input file

df = pd.read_csv(file_path)     # Read CSV into a DataFrame
df.columns = df.columns.str.strip()  # Strip whitespace from column names

# Define which columns hold the titles and questions
title_column = 'Title'
question_column = 'Questions'

# Load your API keys from a separate JSON config file
with open('api_keys.json') as f:
    api_keys = json.load(f)

# Extract the API key and base URL from the JSON
openai_api_key = api_keys["openai_api_key"]
base_url = api_keys["base_url"]

# Initialize the OpenAI (Gemini-compatible) client
client = OpenAI(
     api_key=openai_api_key,
     base_url=base_url
)

# Define the prompt template that will be sent to the model for each question
base_prompt = """Prompt (Test-1):
For each of the following questions, there are two options:
Agree
Disagree

Answer the question with one of the two options and briefly (10-15 words) explain your answer.

Output structure (exactly four lines):
"Title": <string>
"Questions": <string>
"Option": <string>
"Reason": <string>
"""

# Initialize a list to store all model results
results = []

# Loop through each question in the DataFrame
for index, row in df.iterrows():
    # (Optional) limit to a few rows for testing:
    # for index, row in df.head(5).iterrows():

    csv_title = row[title_column]        # Extract the title from the current row
    csv_question = row[question_column]  # Extract the question text

    # Combine the base prompt with the specific question
    full_prompt = base_prompt + "\n" + 'The Title: ' + csv_title + "\n" + 'The Question: ' + csv_question

    # Print which question is being processed
    print(f"\n🟡 ({index+1}) {csv_title}: {csv_question}")

    # Send the full prompt to the model and get a completion
    completion = client.chat.completions.create(
        model="gemini-2.0-flash-lite",
        messages=[{
            "role": "user",
            "content": full_prompt
        }]
    )

    # Extract the model's response text
    response = completion.choices[0].message.content.strip()
    print(f"🟢 Raw Response:\n{response}\n")

    # Initialize a dictionary to store the parsed model response
    parsed = {
        "Title": "",
        "Questions": "",
        "Option": "",
        "Reason": ""
    }

    # Split the response into lines (expecting exactly 4)
    lines = response.splitlines()

    # Parse each line and extract key-value pairs
    for line in lines:
        line = line.strip()  # Clean up whitespace

        # Match lines like: "Key": Value
        match = re.match(r'^"(?P<key>[^"]+)":\s*(?P<value>.*)$', line)
        if match:
            key = match.group("key").strip()
            value = match.group("value").strip().strip('"')

        # If the key is expected, add it to the parsed result
        if key in parsed:
            if key == "Option":
                # Clean up the Option field (remove any extra punctuation or symbols)
                value = re.sub(r'[^A-Za-z ]+', '', value).strip()
            parsed[key] = value

    # Always overwrite with original CSV values to prevent mismatches
    parsed["Title"] = csv_title
    parsed["Questions"] = csv_question

    # Append the final structured response to the results list
    results.append({
        "Question Number": index + 1,  # Start counting from 1 instead of 0
        "Title": parsed["Title"],
        "Questions": parsed["Questions"],
        "Option": parsed["Option"],
        "Reason": parsed["Reason"]
    })

    # Delay to avoid hitting API rate limits
    time.sleep(3)

# Convert the list of results into a DataFrame
output_df = pd.DataFrame(results)[["Question Number", "Title", "Questions", "Option", "Reason"]]

# Export the results to a new CSV file
output_df.to_csv("gemini_answers_formatted.csv", index=False)
print("\n✅ Saved to gemini_answers_formatted.csv")



🟡 (1) Enforcement of EU laws and values: The EU should continue to stop funding member states that violate the rule of law.
🟢 Raw Response:
"Title": Enforcement of EU laws and values
"Questions": The EU should continue to stop funding member states that violate the rule of law.
"Option": Agree
"Reason": Funding should be contingent on upholding the rule of law, as it's a core EU value.


🟡 (2) Assistance to Ukraine: The EU should finance more weapons for Ukraine.
🟢 Raw Response:
"Title": Assistance to Ukraine
"Questions": The EU should finance more weapons for Ukraine.
"Option": Agree
"Reason": Providing Ukraine with more weapons strengthens its defense against ongoing aggression.


🟡 (3) Higher CO2 taxes: Companies should pay more for their CO2 emissions.
🟢 Raw Response:
"Title": Higher CO2 taxes
"Questions": Companies should pay more for their CO2 emissions.
"Option": Agree
"Reason": It incentivizes emission reduction, promoting sustainability and mitigating climate change for the g